In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
pwd

'c:\\Users\\shrin\\Documents\\Python_Scripts\\FetchSearch'

In [30]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    weight_decay: float
    special_tokens: list

In [31]:
from fetchSearch.constants import *
from fetchSearch.utils.common import read_yaml, create_directories

In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
   
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            special_tokens = params.special_tokens,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            weight_decay = params.weight_decay           
        )

        return model_trainer_config

In [33]:
import os
from fetchSearch.logging import logger
from sentence_transformers import SentenceTransformer, models, InputExample, losses, evaluation, util
from datasets import load_dataset, load_from_disk
import torch
from torch.utils.data import DataLoader
import tqdm

In [40]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        sbert_model = SentenceTransformer(self.config.model_ckpt).to(device)
        
        word_embedding_model = sbert_model._first_module()
        word_embedding_model.tokenizer.add_tokens(self.config.special_tokens, special_tokens=True)
        word_embedding_model.auto_model.resize_token_embeddings(len(word_embedding_model.tokenizer))

        loss = losses.CosineSimilarityLoss(model=sbert_model)

        #loading data 
        #dataset_processed = load_from_disk(self.config.data_path)


        train_dataloader = torch.load(os.path.join(self.config.data_path,"train.pth"))#DataLoader(train_examples, shuffle=True, batch_size=16)
        val_dataloader = torch.load(os.path.join(self.config.data_path,"val.pth"))
        test_dataloader = torch.load(os.path.join(self.config.data_path,"test.pth"))

        val_evaluator = torch.load(os.path.join(self.config.data_path,"val_eval.pth"))
        test_evaluator = torch.load(os.path.join(self.config.data_path,"test_eval.pth"))

        sbert_model.fit(train_objectives=[(test_dataloader, loss)], 
                        evaluator=val_evaluator, 
                        epochs = self.config.num_train_epochs,
                        warmup_steps= self.config.warmup_steps, 
                        weight_decay= self.config.weight_decay, 
                        output_path= os.path.join(self.config.root_dir,"saved-search-model")
                    )

In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e